In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions

pipeline_options_dict = {
    'runner': 'DirectRunner',  # Ou 'DataflowRunner' se você estiver executando no Dataflow
    'streaming': True,
}

pipeline_options = PipelineOptions.from_dictionary(pipeline_options_dict)

with beam.Pipeline(options = pipeline_options) as pipeline:
    processing = (
        pipeline
        | beam.io.ReadFromPubSub(subscription='projects/playground-s-11-a3b55282/subscriptions/dataflow').with_output_types(bytes)
        | beam.Map(print)#beam.WindowInto(beam.window.FixedWindows(window_size), trigger=beam.trigger.AfterWatermark(), accumulation_mode=beam.trigger.AccumulationMode.DISCARDING)
        #| beam.io.WriteToText('./test/')
    )


In [2]:
for i in range(3000):
    from time import sleep
    print(i)
    sleep(30)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [ ]:
import argparse
import apache_beam as beam
from apache_beam.transforms.userstate import BagStateSpec
from apache_beam.coders.coders import TupleCoder, FloatCoder
from apache_beam.options.pipeline_options import PipelineOptions

# Constantes
SCHEMA = 'code:STRING,rate:FLOAT,volume:INTEGER,cap:INTEGER,circulatingSupply:INTEGER\
,totalSupply:INTEGER,maxSupply:INTEGER,max_price:FLOAT,min_price:FLOAT,timestamp:TIMESTAMP'
SUBSCRIPTION = 'projects/playground-s-11-7b1242ce/subscriptions/dataflow'

# Funções de transformação
def to_json(data):
    import json
    """Converte uma string JSON em um objeto Python."""
    data = json.loads(data)
    return data

def streaming_columns(crypto_dict):
    """Filtra e renomeia as colunas do dicionário de criptomoedas."""
    return {
        'code':crypto_dict.get('code'),
        'rate':crypto_dict.get('rate'),
        'volume':crypto_dict.get('volume'),
        'cap':crypto_dict.get('cap'),
        'circulatingSupply':crypto_dict.get('circulatingSupply'),
        'totalSupply':crypto_dict.get('totalSupply'),
        'maxSupply':crypto_dict.get('maxSupply')}

class SetCoinKey(beam.DoFn):
    """Define a chave para cada elemento com base no código."""
    def process(self, element, *args, **kwargs):
        yield element['code'], element

class MinMaxBitcoinPriceFn(beam.DoFn):
    """Calcula os preços máximo e mínimo do Bitcoin."""
    PRICE_STATE = BagStateSpec('price_state', TupleCoder((FloatCoder(), FloatCoder())))

    def process(self, element, prev_state=beam.DoFn.StateParam(PRICE_STATE), *args, **kwargs):
        current_price = element[1]['rate']
        previous_prices = list(prev_state.read())

        if previous_prices:
            if current_price > previous_prices[0][0]:
                prev_state.clear()
                prev_state.add((current_price, previous_prices[0][1]))
            elif current_price < previous_prices[0][1]:
                prev_state.clear()
                prev_state.add((previous_prices[0][0], current_price))
        else:
            prev_state.add((current_price, current_price))

        actual_prices = list(prev_state.read())
        element[1]['max_price'] = actual_prices[0][0]
        element[1]['min_price'] = actual_prices[0][1]
        yield element

def format_for_bigquery(element):
    """Formata o elemento para o formato adequado do BigQuery."""
    
    from datetime import datetime
    
    _, data = element
    
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    data_to_bq = {
        'code': data['code'],
        'rate': data['rate'],
        'volume': data['volume'],
        'cap': data['cap'],
        'circulatingSupply': data['circulatingSupply'],
        'totalSupply': data['totalSupply'],
        'maxSupply': data['maxSupply'],
        'max_price': data['max_price'],
        'min_price': data['min_price'],
    }

    data_to_bq['timestamp'] = timestamp

    return data_to_bq

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    known_args = parser.parse_known_args()
    p = beam.Pipeline(options=PipelineOptions())

    (p | beam.io.ReadFromPubSub(subscription=SUBSCRIPTION).with_output_types(bytes)
    
    | beam.Map(lambda x: x.decode('utf-8'))
    | beam.FlatMap(to_json)
    | beam.Map(streaming_columns)
    | beam.ParDo(SetCoinKey())
    | beam.ParDo(MinMaxBitcoinPriceFn())
    | beam.Map(format_for_bigquery)
    | beam.Map(add_timestamp)
    | beam.io.WriteToBigQuery('playground-s-11-7b1242ce:crypto.crypto_price',
                            create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                            write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
                            schema=SCHEMA))
    result = p.run()
    result.wait_until_finish()
